In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from io import StringIO
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from dataset import *
from torch.utils.data import DataLoader

In [3]:
root_folder = 'data/simple_reacher/training'
number_of_trajectories = 100
batch_size = 32

In [39]:
root_folder = 'data/simple_reacher/sample_data_all_limited_joints'
number_of_trajectories = 5

In [41]:
start = time.time()
ondemand = SimpleReacherOnDemandDataset(root_folder, number_of_trajectories)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.07968378067016602 sec


In [5]:
start = time.time()
preloaded = SimpleReacherPreLoadedDataset(root_folder, number_of_trajectories)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 10.221627950668335 sec


In [6]:
ondemand.trajectory_data.shape

(2194, 9)

In [7]:
len(ondemand)

2194

In [8]:
print(preloaded.trajectory_data.shape)
print(preloaded.image_data.shape)

(2194, 9)
(2194, 62500)


In [9]:
idx = np.random.randint(len(ondemand.trajectory_data), size=(batch_size))

In [10]:
idx

array([ 519, 1346,  545,  451,  926, 1596, 1284, 1450,  408,  921, 1548,
       1210,  641,  618, 1954, 1747, 2140, 1758,  242, 1181,  180, 1143,
        334, 1216, 1971, 2152, 2148,  542,   86, 1576,   22,  409])

In [11]:
start = time.time()
ondemandBatch = ondemand[idx]
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.10364007949829102 sec


In [12]:
start = time.time()
preloadedBatch = preloaded[idx]
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.001249074935913086 sec


In [13]:
np.all(ondemandBatch['images'] == preloadedBatch['images'])

True

In [14]:
onedemandLoader = DataLoader(ondemand, batch_size=batch_size, shuffle=True)
preloadedLoader = DataLoader(preloaded, batch_size=batch_size, shuffle=True)

In [15]:
epochs = 1

In [16]:
def simulate_training(dataLoader):
    start = time.time()
    builder = StringIO()
    for e in range(epochs):
        for i, data in enumerate(dataLoader):
            builder.write('Epoch {}: Running batch index = {} with data shape = {}'.format(e, i, data['images'].shape))
            builder.write('\n')
            break
    print ("Time Taken: {} sec".format(time.time() - start))
    log = builder.getvalue()
    builder.close()
    return log

In [17]:
log1 = simulate_training(onedemandLoader)

Time Taken: 0.2524890899658203 sec


In [18]:
log1

'Epoch 0: Running batch index = 0 with data shape = torch.Size([32, 62500])\n'

In [19]:
log2 = simulate_training(preloadedLoader)

Time Taken: 0.005193948745727539 sec


In [20]:
log2

'Epoch 0: Running batch index = 0 with data shape = torch.Size([32, 62500])\n'

In [29]:
sys.getsizeof(ondemand.trajectory_data)

158080

In [27]:
sys.getsizeof(preloaded.image_data)

137125112

In [7]:
start = time.time()
train_data = SimpleReacherOnDemandDataset(root_folder, range(4750), device=None)
test_data = SimpleReacherOnDemandDataset(root_folder, range(4750, 5000), device=None)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 21.97927212715149 sec


In [8]:
print (train_data.trajectory_data.shape)
print (test_data.trajectory_data.shape)

(98892, 9)
(5206, 9)


In [9]:
print (np.min(train_data.trajectory_data[:, :1]))
print (np.max(train_data.trajectory_data[:, :1]))
print (np.min(train_data.trajectory_data[:, 1:3]))
print (np.max(train_data.trajectory_data[:, 1:3]))
print (np.min(train_data.trajectory_data[:, 3:6]))
print (np.max(train_data.trajectory_data[:, 3:6]))
print (np.min(train_data.trajectory_data[:, 6:]))
print (np.max(train_data.trajectory_data[:, 6:]))
print (np.min(test_data.trajectory_data[:, :3]))
print (np.max(test_data.trajectory_data[:, :3]))
print (np.min(test_data.trajectory_data[:, 3:6]))
print (np.max(test_data.trajectory_data[:, 3:6]))
print (np.min(test_data.trajectory_data[:, 6:]))
print (np.max(test_data.trajectory_data[:, 6:]))
print (np.pi)

-3.141592
3.141592
-2.9
2.9
-3.1412612750944597
3.138749504393774
-2.4523708495071115
2.5556591643818747
-3.129891384701949
3.141592
-3.129891384272161
3.0907654726696845
-1.9018395734261253
1.971451623142999
3.141592653589793


In [10]:
actions = train_data.trajectory_data[:, :1]
print (actions.shape)
mask = np.any(actions > 20.0, axis=1)
print (mask.shape)
result = actions[mask]
print (result.shape)
result[:1]

(98892, 1)
(98892,)
(0, 1)


array([], shape=(0, 1), dtype=float64)